In [1]:
] activate .; instantiate; st

Activating environment at `~/Dropbox/Code/CVaR-EconomicDispatch/Project.toml`
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Project CvARED v0.1.0
    Status `~/Dropbox/Code/CVaR-EconomicDispatch/Project.toml`
  [336ed68f]   CSV v0.5.11
  [a93c6f00]   DataFrames v0.19.3
  [31c24e10]   Distributions v0.21.1
  [60bf3e95]   GLPK v0.10.0
  [b6b21f68]   Ipopt v0.5.4
  [4076af6c]   JuMP v0.19.2
  [b8f27783]   MathOptInterface v0.8.4
  [739be429] ↑ MbedTLS v0.6.8 ⚲ ⇒ v0.7.0
  [ab2f91bb]   OSQP v0.5.2
  [c36e90e8] + PowerModels v0.12.4
  [e690365d]   PowerSimulations v0.2.0 [`~/.julia/dev/PowerSimulations`]
  [bcd98974] ~ PowerSystems v0.4.0 #struct_updates (https://github.com/NREL/PowerSystems.jl.git) ⇒ v0.4.0 [`~/.julia/dev/PowerSystems`]
  [295af30f]   Revise v2.1.8
  [9e3dc215]   TimeSeries v0.16.0
  [ade2ca70]   Dates 
  [37e2e46d] + LinearAlgebra 
  [56ddb016]   Logging 
  [9a3f8284]   Random 
  [10745b16]   Stati

In [2]:
include("src/CvARED.jl")
include("data/generate_data.jl")

┌ Info: Recompiling stale cache file /Users/jdlara/.julia/compiled/v1.2/PowerSystems/rr6EX.ji for PowerSystems [bcd98974-b02a-5e2f-9ee0-a103f5c450dd]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /Users/jdlara/.julia/compiled/v1.2/PowerSimulations/ixScC.ji for PowerSimulations [e690365d-45e2-57bb-ac84-44ba829e73c4]
└ @ Base loading.jl:1240


In [3]:
## UC Model Ref
abstract type TestOptModel <: PSI.AbstractOperationModel end
branches = Dict{Symbol, DeviceModel}()
services = Dict{Symbol, PSI.ServiceModel}()

devices = Dict{Symbol, DeviceModel}(:Generators => DeviceModel(PSY.ThermalStandard, PSI.ThermalBasicUnitCommitment),
                                    :Ren => DeviceModel(PSY.RenewableDispatch, PSI.RenewableFullDispatch),
                                    :Loads =>  DeviceModel(PSY.PowerLoad, PSI.StaticPowerLoad),
                                    )       


model_ref_uc= ModelReference(CopperPlatePowerModel, devices, branches, services);

In [4]:
UC_OpModel = OperationModel(TestOptModel, model_ref_uc, sys_uc; optimizer = Gurobi_optimizer)
res = solve_op_model!(UC_OpModel)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 39408 rows, 41976 columns and 110470 nonzeros
Model has 9744 quadratic objective terms
Variable types: 12456 continuous, 29520 integer (29520 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+01]
  Objective range  [2e+02, 4e+03]
  QObjective range [2e+01, 4e+01]
  Bounds range     [1e-04, 3e+02]
  RHS range        [7e-01, 9e+02]
Presolve removed 844 rows and 3414 columns
Presolve time: 0.16s
Presolved: 38564 rows, 38562 columns, 105806 nonzeros
Presolved model has 9744 quadratic objective terms
Variable types: 9862 continuous, 28700 integer (28700 binary)
Found heuristic solution: objective 3.440448e+07
Found heuristic solution: objective 3.355123e+07
Found heuristic solution: objective 3.352435e+07

Root relaxation: objective 3.179704e+07, 49573 iterations, 2.36 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl | 

┌ Warning: Initial status conditions not provided. This can lead to unwanted results
└ @ PowerSimulations ~/.julia/dev/PowerSimulations/src/devices/device_models/thermal_generation.jl:196


Results Model


In [5]:
## ED Model ref
branches = Dict{Symbol, DeviceModel}(#:L => DeviceModel(PSY.Line, PSI.StaticLine),
                                     #:T => DeviceModel(PSY.Transformer2W, PSI.StaticTransformer),
                                     #:TT => DeviceModel(PSY.TapTransformer, PSI.StaticTransformer),
                                     #:dc_line => DeviceModel(PSY.HVDCLine, PSI.HVDCDispatch)
                                        )

services = Dict{Symbol, PSI.ServiceModel}()

devices = Dict{Symbol, DeviceModel}(:Generators => DeviceModel(PSY.ThermalStandard, PSI.ThermalDispatchNoMin, SemiContinuousFF(:P, :ON)),
                                    :Ren => DeviceModel(PSY.RenewableDispatch, PSI.RenewableFullDispatch),
                                    :Loads =>  DeviceModel(PSY.PowerLoad, PSI.StaticPowerLoad),
                                    #:ILoads =>  DeviceModel(PSY.InterruptibleLoad, PSI.DispatchablePowerLoad),
                                    )       

model_ref_ed= ModelReference(CopperPlatePowerModel, devices, branches, services);

In [6]:
ed_OpModel = OperationModel(TestOptModel, model_ref_ed, sys_ed; optimizer = Gurobi_optimizer, parameters = true)
res = solve_op_model!(ed_OpModel)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 14784 rows, 11148 columns and 25920 nonzeros
Model has 4872 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 2e+02]
  QObjective range [2e+00, 3e+00]
  Bounds range     [1e-04, 3e+02]
  RHS range        [1e-02, 8e+02]
Presolve removed 14772 rows and 6216 columns
Presolve time: 0.01s
Presolved: 12 rows, 4932 columns, 4932 nonzeros
Presolved model has 4872 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.200e+01 (roughly 2 MBytes of memory)
 Factor Ops : 1.200e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.60692226e+09 -1.26474659e+09  1.07e+05 8.33e+02  1.00e+06     0s
   1   2.82444359e+06 -1.83757619e+07  4.39e

┌ Warning: The addition operator has been used on JuMP expressions a large number of times. This warning is safe to ignore but may indicate that model generation is slower than necessary. For performance reasons, you should not add expressions in a loop. Instead of x += y, use add_to_expression!(x,y) to modify x in place. If y is a single variable, you may also use add_to_expression!(x, coef, y) for x += coef*y.
└ @ JuMP ~/.julia/packages/JuMP/ibcEh/src/JuMP.jl:692


Results Model


In [63]:
include("src/cvar_network.jl")
include("src/cvar_devices.jl")

In [64]:
branches = Dict{Symbol, DeviceModel}(#:L => DeviceModel(PSY.Line, PSI.StaticLine),
                                     #:T => DeviceModel(PSY.Transformer2W, PSI.StaticTransformer),
                                     #:TT => DeviceModel(PSY.TapTransformer, PSI.StaticTransformer),
                                     #:dc_line => DeviceModel(PSY.HVDCLine, PSI.HVDCDispatch)
                                        )

services = Dict{Symbol, PSI.ServiceModel}()

devices = Dict{Symbol, DeviceModel}(:Generators => DeviceModel(PSY.ThermalStandard, PSI.ThermalDispatchNoMin, SemiContinuousFF(:P, :ON)),
                                    :Ren => DeviceModel(PSY.RenewableDispatch, RenewableCvAR),
                                    :Loads =>  DeviceModel(PSY.PowerLoad, PSI.StaticPowerLoad),
                                    #:ILoads =>  DeviceModel(PSY.InterruptibleLoad, PSI.DispatchablePowerLoad),
                                    )       

model_ref_cvar= ModelReference(CVaRModel, devices, branches, services);

In [65]:
cvar_OpModel = OperationModel(TestOptModel, model_ref_cvar, sys_cvar; optimizer = Gurobi_optimizer, parameters = true)

Academic license - for non-commercial use only


Operation Model


In [66]:
res = solve_op_model!(cvar_OpModel)

Academic license - for non-commercial use only
Optimize a model with 14808 rows, 12264 columns and 34344 nonzeros
Model has 4908 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [6e+01, 2e+02]
  QObjective range [2e+00, 1e+08]
  Bounds range     [1e-02, 1e+01]
  RHS range        [1e-02, 8e+02]
Presolve removed 14760 rows and 4920 columns
Presolve time: 0.02s
Presolved: 48 rows, 7344 columns, 14664 nonzeros
Presolved model has 4908 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 12
 AA' NZ     : 6.000e+01
 Factor NZ  : 1.800e+02 (roughly 3 MBytes of memory)
 Factor Ops : 6.600e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.60704334e+09 -1.26496004e+09  1.24e+05 8.33e+02  1.00e+06     0s
   1   3.07713135e+07 -2.44113311e+07  8.20e+03 5.46e+01  6.70e+04     0s

BoundsError: BoundsError: attempt to access (1:12,)
  at index [2]

In [74]:
variable = PSI.var(cvar_OpModel.canonical, :P_ThermalStandard)
result = Array{Float64, length(variable.axes)}(undef, length(variable.axes[2]), length(variable.axes[1]))
for t in variable.axes[2], (ix, name) in enumerate(variable.axes[1])
        result[t, ix] = JuMP.value(variable[name, t])
end

In [69]:
cvar_OpModel.canonical.variables

Dict{Symbol,JuMP.Containers.DenseAxisArray} with 5 entries:
  :P_ThermalStandard => 2-dimensional DenseAxisArray{VariableRef,2,...} with in…
  :delta_rhs         => 1-dimensional DenseAxisArray{VariableRef,1,...} with in…
  :u_rhs             => 2-dimensional DenseAxisArray{VariableRef,2,...} with in…
  :delta_lhs         => 1-dimensional DenseAxisArray{VariableRef,1,...} with in…
  :u_lhs             => 2-dimensional DenseAxisArray{VariableRef,2,...} with in…

In [75]:
result

12×410 Array{Float64,2}:
 2.3505  4.6914e-12   1.7267  1.149  …  0.05  0.605  1.63  8.15323  1.106
 2.3505  4.26971e-12  1.7267  1.149     0.05  0.605  1.63  6.47629  1.106
 2.3505  4.1745e-12   1.7267  1.149     0.05  0.605  1.63  6.05076  1.106
 2.3505  4.09398e-12  1.7267  1.149     0.05  0.605  1.63  5.67549  1.106
 2.3505  4.03744e-12  1.7267  1.149     0.05  0.605  1.63  5.40301  1.106
 2.3505  4.31219e-12  1.7267  1.149  …  0.05  0.605  1.63  6.6601   1.106
 2.3505  4.72491e-12  1.7267  1.149     0.05  0.605  1.63  8.27365  1.106
 2.3505  5.16562e-12  1.7267  1.149     0.05  0.605  1.63  9.71167  1.106
 2.3505  5.01103e-12  1.7267  1.149     0.05  0.605  1.63  9.25593  1.106
 2.3505  4.92178e-12  1.7267  1.149     0.05  0.605  1.63  8.94787  1.106
 2.3505  4.8415e-12   1.7267  1.149  …  0.05  0.605  1.63  8.67954  1.106
 2.3505  4.91882e-12  1.7267  1.149     0.05  0.605  1.63  8.9381   1.106